This notebook exists to experiment with different methods for classification

In [1]:
%run "Feature_Selection.ipynb" 

100%|███████████████████████████████████████████████████████████████████████████████| 223/223 [00:01<00:00, 121.80it/s]


In [2]:
removed_params = ['rr_std', 'RMSSD', 'pNN50', 'mean', 'lf', 'hf', 'skews', 'rr_mean', 'power_ratio', 'std', 'kurtosis', 'shannon_en'] #convert to a set?

selected_params = {}
for key in params.keys():
    if key not in removed_params:
        selected_params[key] = params[key]
print(selected_params.keys())

dict_keys(['sd_ratio'])


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [4]:
health_state = np.array(allowed_patients.get_diagnoses())

encoded_health_state = [1 if label == 'Unhealthy' else -1 for label in health_state]

print(len(health_state))

223


In [5]:
def get_balanced_accuracy(y_test, y_pred):
    """
    balanced accuracy....
    """
    num_healthy_true = np.sum([x=='Healthy' for x in y_test])
    num_unhealthy_true = len(y_test) - num_healthy_true
    count_healthy_accurate = 0
    count_unhealthy_accurate = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Unhealthy':
            count_unhealthy_accurate +=1
        elif y_pred[i] == y_test[i] == 'Healthy':
            count_healthy_accurate +=1
    healthy_percentage = count_healthy_accurate/num_healthy_true
    unhealthy_percentage = count_unhealthy_accurate/num_unhealthy_true
    balanced_accuracy = (healthy_percentage + unhealthy_percentage) * 0.5
    return balanced_accuracy

def get_specificity(y_test, y_pred):
    """
    true negative rate
    """
    true_negative = 0
    false_positive = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Healthy':
            true_negative += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Healthy':
            false_positive += 1
    
    return true_negative / (true_negative+false_positive)
            
def get_f1_score(y_test, y_pred):
    """
    balance between precision and recall
    """
    true_positive = 0
    false_positive = 0
    false_negative = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Unhealthy':
            true_positive += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Healthy':
            false_positive += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Unhealthy':
            false_negative += 1
    return (2*true_positive)/(2*true_positive+false_positive+false_negative)


def get_av_confusion_matrix(y_test, y_pred):
    av_confusion_mat = np.zeros(shape = (len(y_test), 2, 2))
    for i in range(0, len(y_test)):
        av_confusion_mat[i] = confusion_matrix(y_test[i], y_pred[i])
    return np.mean(av_confusion_mat, axis=0)

def scoring_function(model, X, y):
    """
    change to incorporate balanced accuracy 
    """
    
    y_pred = model.predict(X)
    y_test = y
    balanced_acc = get_balanced_accuracy(y_test, y_pred)
    specificity = get_specificity(y_test, y_pred)
    f1 = get_f1_score(y_test, y_pred)
    
    
    return f1*0.7 + balanced_acc*0.3

## SVM

In [6]:
def convert_dict_to_array(selected_params):
    no_features = len(selected_params)
    selected_params_array = np.zeros((no_patients, no_features))
    
    for i, values in enumerate(selected_params.values()):
        selected_params_array[:, i] = values
    return selected_params_array, no_features

        
selected_params_array, no_features = convert_dict_to_array(selected_params)


### Basic SVM

In [7]:
#splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(selected_params_array, health_state, test_size=0.3, stratify=health_state)

#init and train model, using radial basis functions
svm_model = SVC(kernel='rbf', gamma='scale', probability=True, class_weight = 'balanced')  #'scale' normalises data, prevents overfitting
#probability = True allows calculation of probabilities through 5-fold CV

svm_model.fit(X_train, y_train)

#predicting probabilities
probabilities = svm_model.predict_proba(X_test)
print(probabilities)


#predictions
y_pred = svm_model.predict(X_test)

#evaluating accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = get_balanced_accuracy(y_test, y_pred)
print("Accuracy:", accuracy)
print('balanced accuracy:', balanced_accuracy)

print(np.sum([x=='Healthy' for x in y_test]))
print(y_test, y_pred)

#view a classification report
# print("Classification Report:")
# print(classification_report(y_test, y_pred))

[[0.13363    0.86637   ]
 [0.29630688 0.70369312]
 [0.28988466 0.71011534]
 [0.20742053 0.79257947]
 [0.2796396  0.7203604 ]
 [0.1361378  0.8638622 ]
 [0.29165357 0.70834643]
 [0.1356621  0.8643379 ]
 [0.26945059 0.73054941]
 [0.15920008 0.84079992]
 [0.1316341  0.8683659 ]
 [0.22802901 0.77197099]
 [0.28682479 0.71317521]
 [0.23465634 0.76534366]
 [0.25320569 0.74679431]
 [0.28715428 0.71284572]
 [0.23507011 0.76492989]
 [0.294684   0.705316  ]
 [0.30170455 0.69829545]
 [0.30368778 0.69631222]
 [0.30389297 0.69610703]
 [0.14167808 0.85832192]
 [0.30413642 0.69586358]
 [0.17810284 0.82189716]
 [0.22019035 0.77980965]
 [0.1336455  0.8663545 ]
 [0.28884694 0.71115306]
 [0.19511214 0.80488786]
 [0.30219538 0.69780462]
 [0.13415345 0.86584655]
 [0.22230307 0.77769693]
 [0.13611391 0.86388609]
 [0.16358723 0.83641277]
 [0.14912911 0.85087089]
 [0.27510598 0.72489402]
 [0.30400168 0.69599832]
 [0.21911312 0.78088688]
 [0.15941062 0.84058938]
 [0.15674801 0.84325199]
 [0.16907395 0.83092605]


### Hyper Parameter Tuning
- c is inversely proportional to l2 regularisation parameter
- gamma determines scaling of features


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score

#recall needs encoded_health_state
#splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(selected_params_array, health_state, test_size=0.3, stratify=health_state)

# define hyperparameter grid to test
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

#initialise
svc = SVC(class_weight='balanced')

# perform grid search
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring=scoring_function)
grid_search.fit(X_train, y_train)


print(grid_search.best_params_)

# train with best parameters
best_svc = grid_search.best_estimator_
best_svc.fit(X_train, y_train)

{'C': 0.01, 'gamma': 'scale', 'kernel': 'linear'}


SVC(C=0.01, class_weight='balanced', kernel='linear')

In [9]:
#predictions
y_pred = best_svc.predict(X_test)

#evaluating accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = get_balanced_accuracy(y_test, y_pred)

print("Accuracy:", accuracy)
print('balanced accuracy:', balanced_accuracy)

Accuracy: 0.208955223880597
balanced accuracy: 0.5


### Stratified k-fold validation

need to weight the train and test sets to cope with difference with numbers of healthy and unhealthy, stratified k-fold validation

In [10]:
from sklearn.model_selection import StratifiedKFold

n_splits = 3 #3 fold validation
skf = StratifiedKFold(n_splits=n_splits, shuffle=True) #can do repeated skf for better validation


accuracies=[]
balanced_accuracies=[]
y_pred_list = []
y_test_list=[]
for train_index, test_index in skf.split(selected_params_array, health_state):
    #getting test and train data sets
    X_train, X_test = selected_params_array[train_index], selected_params_array[test_index]
    y_train, y_test = health_state[train_index], health_state[test_index]
    
    y_test_list.append(y_test)
    #training model with tuned hyperparameters
    tuned_svc = grid_search.best_estimator_
    tuned_svc.fit(X_train, y_train)
    
    #evaluating model
    y_pred = tuned_svc.predict(X_test)
    y_pred_list.append(y_pred)
    accuracies.append(accuracy_score(y_test, y_pred))
    balanced_accuracies.append((get_balanced_accuracy(y_test, y_pred)))

In [11]:
print(accuracies)
print(balanced_accuracies)

[0.6933333333333334, 0.7972972972972973, 0.7972972972972973]
[0.4406779661016949, 0.5, 0.5]


In [12]:
print(np.mean(balanced_accuracies))

0.48022598870056493


In [13]:
print(y_pred_list[2], y_test_list[2])
print(confusion_matrix(y_pred_list[2], y_test_list[2]))

['Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy'] ['Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 

In [14]:
print(y_pred, y_test)

['Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy'] ['Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 

In [15]:
print("Classification Report:")
print(classification_report(y_test_list[1], y_pred_list[1]))
print(get_specificity(y_test_list[1], y_pred_list[1]))
print(get_av_confusion_matrix(y_test_list, y_pred_list))

Classification Report:
              precision    recall  f1-score   support

     Healthy       0.00      0.00      0.00        15
   Unhealthy       0.80      1.00      0.89        59

    accuracy                           0.80        74
   macro avg       0.40      0.50      0.44        74
weighted avg       0.64      0.80      0.71        74

0.0
[[ 0.         15.33333333]
 [ 2.33333333 56.66666667]]


C:\Users\court\Documents\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\court\Documents\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\court\Documents\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
